# Import Packages

In [ ]:
import sys
sys.path.append('../input/timm-pytorch-image-models/pytorch-image-models-master')
import timm

In [ ]:
# model = timm.create_model('eca_nfnet_l0', pretrained=True)


In [ ]:
# !pip install torchsummary 
# from torchsummary import summary
# summary(model,(3, 512,512))

In [ ]:
import os
import cv2
import math
import random
import numpy as np
import pandas as pd
from tqdm import tqdm

import albumentations
from albumentations.pytorch.transforms import ToTensorV2

import torch
import timm
import torch
import torch.nn as nn
from torch.nn import functional as F
from torch.utils.data import Dataset,DataLoader

import gc
import matplotlib.pyplot as plt
import cudf
import cuml
import cupy
from cuml.feature_extraction.text import TfidfVectorizer
from cuml import PCA
from cuml.neighbors import NearestNeighbors

# Config

In [ ]:
class CFG:
    seed = 54
    classes = 11014 
    scale = 30 
    margin = 0.5
    model_name =  'tf_efficientnet_b3'
    fc_dim = 512
    img_size = 512
    batch_size = 20
    num_workers = 4
    device = device = 'cuda' if torch.cuda.is_available() else 'cpu'
    model_path = '../input/shopee-correct-label-train/arcface_512x512_tf_efficientnet_b3_14_epoch_correct_label.pt'
#     model_path = ['../input/shopee-match-ensemble-eff-b1-b3-focal-loss/B1arcface_512x512_tf_efficientnet_b1_nofold_14_epo.pt',
#                   '../input/shopee-training-eff-b4/arcface_512x512_tf_efficientnet_b2_14_epoch.pt',
#                   '../input/shopee-match-ensemble-eff-b1-b3-focal-loss/arcface_512x512_tf_efficientnet_b3_14_epoch.pt']
#     model_path = ['../input/eff-b4-tfidf/arcface_512x512_tf_efficientnet_b4_0_fold_15_epoch.pt',
#                  '../input/eff-b4-tfidf/arcface_512x512_tf_efficientnet_b4_1_fold_15_epoch.pt',
#                  '../input/eff-b4-tfidf/arcface_512x512_tf_efficientnet_b4_2_fold_15_epoch.pt']

# Utils

In [ ]:
def read_dataset():
    df = pd.read_csv('../input/shopee-product-matching/test.csv')
    df_cu = cudf.DataFrame(df)
    image_paths = '../input/shopee-product-matching/test_images/' + df['image']
        
    return df, df_cu, image_paths
 
     

In [ ]:
def seed_torch(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    
seed_torch(CFG.seed)

# Verify the CV Score of image_hase

In [ ]:
# df = pd.read_csv('../input/shopee-product-matching/train.csv')
# df_cu = cudf.DataFrame(df)
# image_paths = '../input/shopee-product-matching/train_images/' + df['image']

In [ ]:
# tmp = df.groupby('image_phash').posting_id.agg('unique').to_dict()
# df['oof'] = df.image_phash.map(tmp)
# df.head()

In [ ]:
# tmp = df.groupby('label_group').posting_id.agg('unique').to_dict()
# df['target'] = df.label_group.map(tmp)
# df.head()

# F1 score

In [ ]:
def getMetric(col):
    def f1score(row):
        n = len( np.intersect1d(row.target,row[col]) )
        return 2*n / (len(row.target)+len(row[col]))
    return f1score

# df['f1'] = df.apply(getMetric('oof'),axis=1)
# print('CV score =',df.f1.mean())

# Image Predictions

In [ ]:
# Create Model


class ArcMarginProduct(nn.Module):
    def __init__(self, in_features, out_features, scale=30.0, margin=0.50, easy_margin=False, ls_eps=0.0):
        super(ArcMarginProduct, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.scale = scale
        self.margin = margin
        self.ls_eps = ls_eps
        self.weight = nn.Parameter(torch.FloatTensor(out_features, in_features))
        nn.init.xavier_uniform_(self.weight)

        self.easy_margin = easy_margin
        self.cos_m = math.cos(margin)
        self.sin_m = math.sin(margin)
        self.th = math.cos(math.pi - margin)
        self.mm = math.sin(math.pi - margin) * margin

    def forward(self, input, label):
        cosine = F.linear(F.normalize(input), F.normalize(self.weight))
        sine = torch.sqrt(1.0 - torch.pow(cosine, 2))
        phi = cosine * self.cos_m - sine * self.sin_m
        if self.easy_margin:
            phi = torch.where(cosine > 0, phi, cosine)
        else:
            phi = torch.where(cosine > self.th, phi, cosine - self.mm)
    
        one_hot = torch.zeros(cosine.size(), device='cuda')
        one_hot.scatter_(1, label.view(-1, 1).long(), 1)
        if self.ls_eps > 0:
            one_hot = (1 - self.ls_eps) * one_hot + self.ls_eps / self.out_features

        output = (one_hot * phi) + ((1.0 - one_hot) * cosine)
        output *= self.scale

        return output, nn.CrossEntropyLoss()(output,label)


class ShopeeModel(nn.Module):

    def __init__(
        self,
        n_classes = CFG.classes,
        model_name = CFG.model_name,
        fc_dim = CFG.fc_dim,
        margin = CFG.margin,
        scale = CFG.scale,
        use_fc = True,
        pretrained = True,
        arc_mode = 0,
        mode = False):

        super(ShopeeModel,self).__init__()
        print('Building Model Backbone for {} model'.format(model_name))

        self.training = mode
        self.backbone = timm.create_model(model_name, pretrained=pretrained)
        in_features = self.backbone.classifier.in_features
        self.backbone.classifier = nn.Identity()
        self.backbone.global_pool = nn.Identity()
        self.pooling =  nn.AdaptiveAvgPool2d(1)
        self.use_fc = use_fc

        if use_fc:
            self.dropout = nn.Dropout(p=0.1)
            self.classifier = nn.Linear(in_features, fc_dim)
            self.bn = nn.BatchNorm1d(fc_dim)
            self._init_params()
            in_features = fc_dim

        self.final = ArcMarginProduct(
            in_features,
            n_classes,
            scale = scale,
            margin = margin,
            easy_margin = False,
            ls_eps = 0.0
        )

    def _init_params(self):
        nn.init.xavier_normal_(self.classifier.weight)
        nn.init.constant_(self.classifier.bias, 0)
        nn.init.constant_(self.bn.weight, 1)
        nn.init.constant_(self.bn.bias, 0)

    def forward(self, image, label):
        features = self.extract_features(image)
        if self.training:
            logits = self.final(features, label)
            return logits
        else:
            return features

    def extract_features(self, x):
        batch_size = x.shape[0]
        x = self.backbone(x)
        x = self.pooling(x).view(batch_size, -1)

        if self.use_fc and self.training:
            x = self.dropout(x)
            x = self.classifier(x)
            x = self.bn(x)
        return x

In [ ]:
def get_image_neighbors(df, embeddings, KNN=50):

    model = NearestNeighbors(n_neighbors = KNN)
    model.fit(embeddings)
    distances, indices = model.kneighbors(embeddings)
    
#     threshold = 0.3
    threshold = 6.4
    predictions = []
    for k in tqdm(range(embeddings.shape[0])):
        idx = np.where(distances[k,] < threshold)[0]
        ids = indices[k,idx]
        posting_ids = df['posting_id'].iloc[ids].values
        predictions.append(posting_ids)
        
    del model, distances, indices
    gc.collect()
    return df, predictions

In [ ]:
def get_test_transforms():
    return albumentations.Compose([
        albumentations.Resize(CFG.img_size, CFG.img_size, always_apply=True),
        albumentations.Normalize(),
        ToTensorV2(p=1.0)
    ])

In [ ]:
class ShopeeDataset(Dataset):

    def __init__(self, image_paths, transforms=None):
        self.image_paths = image_paths
        self.augmentations = transforms

    def __len__(self):
        return self.image_paths.shape[0]

    def __getitem__(self, index):
        image_path = self.image_paths[index]
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        if self.augmentations:
            augmented = self.augmentations(image=image)
            image = augmented['image']
        
        return image, torch.tensor(1)

In [ ]:
def get_image_embeddings(image_paths):

    model = ShopeeModel(pretrained=False).to(CFG.device)
    model.load_state_dict(torch.load(CFG.model_path))
    model.eval()
    # Choose one of fold model to load weigths
#     if fold == 1:
#         checkpoint = torch.load(CFG.model_path[0])
#         checkpoint1 = {k[7:]:v for k, v in checkpoint.items()}
#         model.load_state_dict(checkpoint1)
#         model.eval()
#     elif fold == 2:
#         checkpoint = torch.load(CFG.model_path[1])
# #         checkpoint1 = {k[7:]:v for k, v in checkpoint.items()}
#         model.load_state_dict(checkpoint)
#         model.eval()
#     elif fold == 3:
#         checkpoint = torch.load(CFG.model_path[2])
# #         checkpoint1 = {k[7:]:v for k, v in checkpoint.items()}
#         model.load_state_dict(checkpoint)
#         model.eval()
#     elif fold == 3:
#         model.load_state_dict(torch.load(CFG.model_path[fold]))
#         model.eval()
#      elif fold == 4:
#         model.load_state_dict(torch.load(CFG.model_path[fold]))
#         model.eval()
#     else:
#         model.load_state_dict(torch.load(CFG.model_path[fold]))
#         model.eval()
        
    image_dataset = ShopeeDataset(image_paths=image_paths, transforms=get_test_transforms())
    image_loader = torch.utils.data.DataLoader(
        image_dataset,
        batch_size=CFG.batch_size,
        num_workers=CFG.num_workers
    )

    embeds = []
    with torch.no_grad():
        for img,label in tqdm(image_loader): 
            img = img.cuda()
            label = label.cuda()
            features = model(img,label)
            image_embeddings = features.detach().cpu().numpy()
            embeds.append(image_embeddings)

    del model
    image_embeddings = np.concatenate(embeds)
    print(f'Our image embeddings shape is {image_embeddings.shape}')
    del embeds
    gc.collect()
    return image_embeddings

In [ ]:
df,df_cu,image_paths = read_dataset()

In [ ]:
# Get neighbors for image_embeddings
# image_embeddings1 = get_image_embeddings(image_paths.values, fold=1, arc_mode=0)
# image_embeddings2 = get_image_embeddings(image_paths.values, fold=2, arc_mode=1)
# image_embeddings3 = get_image_embeddings(image_paths.values, fold=3, arc_mode=2)
# image_embeddings = np.concatenate([image_embeddings1, image_embeddings2, image_embeddings3],axis=1)
image_embeddings = get_image_embeddings(image_paths.values)

# image_embeddings = get_image_embeddings(image_paths.values)
df, image_predictions = get_image_neighbors(df, image_embeddings, KNN=50 if len(df)>3 else 3)
df.head()

# Calcaluate the cv score of training dataset

In [ ]:
# df,df_cu,image_paths = read_dataset()

# tmp = df.groupby('label_group').posting_id.agg('unique').to_dict()
# df['target'] = df.label_group.map(tmp)
# df.head()

In [ ]:
# df, image_predictions = get_image_neighbors(df, image_embeddings, KNN=50 if len(df)>3 else 3)
# df['Image_predictions'] = image_predictions
# df['F1'] = df.apply(getMetric('Image_predictions'),axis=1)

# print('F1 Score:{}'.format(df.F1.mean()))
# df.head()

# Calculate the test set

In [ ]:
# def format_convert(match):
#     image_predictions = []
#     for i in match:
#         image_predictions.append( ' '.join((i)))
    
#     return image_predictions
# #         print(image_predictions)

In [ ]:
# df, image_predictions = get_image_neighbors(df, image_embeddings, KNN=50 if len(df)>3 else 3)
# image_predictions = format_convert(image_predictions)

# df['matches'] = image_predictions
# df.head()

# Text Predictions

In [ ]:
def get_text_predictions(df, max_features=25_000):
    
    model = TfidfVectorizer(stop_words='english',
                            binary=True,
                            max_features=max_features)
    text_embeddings = model.fit_transform(df_cu['title']).toarray()

    print('Finding similar titles...')
    CHUNK = 1024 * 4
    CTS = len(df) // CHUNK
    if (len(df)%CHUNK) != 0:
        CTS += 1

    preds = []
    for j in range( CTS ):
        a = j * CHUNK
        b = (j+1) * CHUNK
        b = min(b, len(df))
        print('chunk', a, 'to', b)

        # COSINE SIMILARITY DISTANCE
        cts = cupy.matmul(text_embeddings, text_embeddings[a:b].T).T
        for k in range(b-a):
            IDX = cupy.where(cts[k,]>0.75)[0]
            o = df.iloc[cupy.asnumpy(IDX)].posting_id.values
            preds.append(o)

    del model,text_embeddings
    gc.collect()
    return preds

In [ ]:
text_predictions = get_text_predictions(df, max_features=25_000)

In [ ]:
def combine_predictions(row):
    x = np.concatenate([row['image_predictions'], row['text_predictions']])
    return ' '.join( np.unique(x) )

# Preparing Submissions

In [ ]:
# df[['posting_id', 'matches']].to_csv('submission.csv', index=False)
# a = pd.read_csv('./submission.csv')
# a.head()

In [ ]:
df['image_predictions'] = image_predictions
df['text_predictions'] = text_predictions
df['matches'] = df.apply(combine_predictions, axis=1)
df[['posting_id', 'matches']].to_csv('submission.csv', index=False)

In [ ]:
# def combine_predictions(row):
#     x = np.concatenate([row['image_predictions'], row['text_predictions']])
#     return ' '.join( np.unique(x) )